Step 1. Use one of the deep learning frameworks (Pytorch, Tensorflow, Keras, ...) and
load the pre-trained AlexNet model. Input these 156 images to the pre-trained AlexNet
model and extract feature maps/activations from Conv 1, 2, 3, 4, 5, fc6, and fc7 layers.
Vectorize the activations corresponding to each image. You should have a vector of
activations per image per layer mentioned above. (20 points)

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

Step 2. Create a representational dissimilarity matrix (RDM) which is 156 x156 matrix,
each row and column in this matrix is indexed by one of the images in the image set
and each element in the matrix is the Euclidean distance between the activation vectors
of the corresponding images you extracted in Step 1. (30 points)

Step 3. Plot the RDM for each layer (Conv 1, 2, 3, 4, 5, fc6, fc7), and their
corresponding multidimensional scaling (MDS) visualization in 2 Dimension. The class
labels you should use for the MDS plotting includes images 1 to 28 are Animals, 29-64
are Objects, 65 to 100 are scenes, 101 to 124 are human activities, 125 to 156 are
faces. (30 points)

Please write a short report presenting and discussing the results of this assignment. (20
points)